# **Data_ETL**

## Objectives

* Extract, transform and load data ready for analysis

## Inputs

* Healthcare Insurance data from Kaggle (see README for details)

## Outputs

* Cleaned data for use in analysis and visualisation to be performed undeer Analysis.ipynb


# 1 Import packages

Import relevant packages required for data analysis and visualisation

Import numpy, etc

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# 2 Data extract and familiarisation

Read in csv file and familiarise with the file structure at a high level

In [2]:
df = pd.read_csv('../data/inputs/raw/insurance_data.csv')
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [3]:
df.shape

(1338, 7)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [5]:
df.describe(include = 'all')

,age,sex,bmi,children,smoker,region,charges
count,1338.000000,1338,1338.000000,1338.000000,1338,1338,1338.000000
unique,NaN,2,NaN,NaN,2,4,NaN
top,NaN,male,NaN,NaN,no,southeast,NaN
freq,NaN,676,NaN,NaN,1064,364,NaN
mean,39.207025,NaN,30.663397,1.094918,NaN,NaN,13270.422265
std,14.049960,NaN,6.098187,1.205493,NaN,NaN,12110.011237
min,18.000000,NaN,15.960000,0.000000,NaN,NaN,1121.873900
25%,27.000000,NaN,26.296250,0.000000,NaN,NaN,4740.287150
50%,39.000000,NaN,30.400000,1.000000,NaN,NaN,9382.033000
75%,51.000000,NaN,34.693750,2.000000,NaN,NaN,16639.912515


In [6]:
df['sex'].unique()

array(['female', 'male'], dtype=object)

In [7]:
df['smoker'].unique()

array(['yes', 'no'], dtype=object)

In [8]:
df['region'].unique()

array(['southwest', 'southeast', 'northwest', 'northeast'], dtype=object)

### Findings:
There are 1338 rows of data entries and 7 data fields (age, sex, BMI, number of children, smoker status, region, charges).

Data types all look ok, e.g. age data type is set as integer as expected.

There is no missing data.

Range of values under most data fields look reasonable:

1) Age range between 18 and 64;
2) Two gender types (male and female);
3) Number of children between 0 and 5;
4) Two smoker statuses (yes and no);
5) Four regions (southwest, southeast, northwest and northeast)

Value ranges for BMI and charges don't look too unreasonable either, but we will examine further to see if thee are outliers in our analysis later.

# Data quality check

In addition to checks on data type and missing data above, duplicates and (high level) outlier checks are performed below.

In [9]:
df.duplicated().sum()

1

There is one duplicate, and let's check which rows they are:

In [10]:
df[df.duplicated(keep = False)]

,age,sex,bmi,children,smoker,region,charges
195,19,male,30.59,0,no,northwest,1639.5631
581,19,male,30.59,0,no,northwest,1639.5631


Rows 195 and 581 are identical in every column, and without additional information, there is a good chance that one of them was duplicated.

# Data cleaning and preparation

Decision and rationale for data manipulation:
We would normally seek to consult with the data manager to get a better understanding of the data.  For the purpose of this exercise, we assume that they are duplicated and shall remove one of the records.

Data preparation for analysis:
1) Add a new column to group BMI into categories
2) Create a csv file for the cleaned data

Remove duplicated record and quick check on the number of remaining records:

In [11]:
df_dup_remove = df.drop_duplicates()
df_dup_remove.shape

(1337, 7)

Check if any duplicate remains:

In [12]:
df_dup_remove.duplicated().sum()

0

In [13]:
df_dup_remove.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


There are 6 BMI standard categories.  We use a simplified version below for this exercise and will expand to the full standard categorisation if further breakdown is deemed to be necessary for enhanced assessment and if time permits.

Underweight:   0 <= BMI < 18.5
Normal weight: 18.5 <= BMI < 25
Overweight:    25 <= BMI < 30
Obese:         BMI >= 30

Take a copy of the cleaned dataframe.
Create a dataframe with an added column "weight_group" for the above BMI categorisation.

In [14]:
def weight_grouping(x):
    if x < 18.5:
        return 'Underweight'
    elif x < 25:
        return 'Normal_weight'
    elif x < 30:
        return 'Overweight'
    else:
        return 'Obese'

df_withweightgroup = df_dup_remove.copy()
df_withweightgroup['weight_group'] = df_withweightgroup['bmi'].apply(weight_grouping)
df_withweightgroup.head()

,age,sex,bmi,children,smoker,region,charges,weight_group
0,19,female,27.900,0,yes,southwest,16884.92400,Overweight
1,18,male,33.770,1,no,southeast,1725.55230,Obese
2,28,male,33.000,3,no,southeast,4449.46200,Obese
3,33,male,22.705,0,no,northwest,21984.47061,Normal_weight
4,32,male,28.880,0,no,northwest,3866.85520,Overweight


Save the cleaned and modified into a new csv file:

In [18]:
df_withweightgroup.to_csv('..\data\inputs\cleaned\insurance_data_cleaned.csv', index = False)

<>:1: SyntaxWarning:

invalid escape sequence '\d'

<>:1: SyntaxWarning:

invalid escape sequence '\d'

C:\Users\L30SC\AppData\Local\Temp\ipykernel_14192\2708588581.py:1: SyntaxWarning:

invalid escape sequence '\d'



Data is ready for use for analysis.  This is to be performed under Analysis.ipynb